In [ ]:
import json
import pandas as pd
from datetime import datetime, timezone
import os
import pickle

In [ ]:
def test_ash():
    print("test")


news_path = "/Users/k2rth1k/swm/News"
ignore_files = ['.DS_Store']
news_folders = os.listdir(news_path)
article_content = []
published_time = []
for sub_folder in news_folders:
    if sub_folder in ignore_files:
        continue
    articles = os.listdir(news_path + "/" + sub_folder)
    for article in articles:
        article_path = news_path + "/" + sub_folder + "/" + article
        article_json = open(article_path)
        data = json.load(article_json)
        time_of_publish = data['published']
        time = datetime.fromisoformat(time_of_publish).astimezone(tz=timezone.utc).replace(tzinfo=None)
        published_time.append(time)
        article_content.append(data['text'].lower())

raw_data = pd.DataFrame({
    "published_time": published_time,
    'article_content': article_content
})
print("done")

In [ ]:
apple_news_data = raw_data[raw_data['article_content'].str.contains('apple|aapl')]
amazon_news_data = raw_data[raw_data['article_content'].str.contains("amazon|amzn")]


In [ ]:
amazon_news_data

In [ ]:
raw_data

In [ ]:
with open('pkls/amazon_news_data.pkl', 'wb') as f:
    pickle.dump(amazon_news_data, f)
    print("done")
with open('pkls/apple_news_data.pkl', 'wb') as f:
    pickle.dump(apple_news_data, f)
    print("done")